#Activity: Feature Engineering on Financial Dataset

You are working for a major bank in Czech Republic and you have tasked to analyse the transactions of existing customers. The data team have extracted all the tables from their database they think will be useful for you. You will need to consolidate the data from these tables into a single DataFrame and create new features before analysing customer's behaviors. 

You will be using only the following 4 tables: trans, disp, account and client.

The following steps will help you complete this activity:
- Download and load the different tables from this dataset into Python
- Merge the tables together
- Check results after merging
- Transform data type for date columns
- Fix data quality issue
- Create new feature using date operations

The dataset was originally shared by Berka, Petr for the Discovery Challenge PKDD'99:
https://relational.fit.cvut.cz/dataset/Financial

The CSV version can be found here:
https://data.world/lpetrocelli/czech-financial-dataset-real-anonymized-transactions


1. Open on a new Colab notebook and import the pandas package

In [0]:
import pandas as pd

2. Assign the links to the disp, trans, account and client tables from the Financial dataset to 4 new variables respectively called 'disp_url', 'trans_url', 'account_url' and 'client_url'

In [0]:
disp_url = 'https://raw.githubusercontent.com/PacktWorkshops/The-Data-Science-Workshop/master/Chapter12/Dataset/disp.csv'
trans_url = 'https://raw.githubusercontent.com/PacktWorkshops/The-Data-Science-Workshop/master/Chapter12/Dataset/trans.csv'
account_url = 'https://raw.githubusercontent.com/PacktWorkshops/The-Data-Science-Workshop/master/Chapter12/Dataset/account.csv'
client_url = 'https://raw.githubusercontent.com/PacktWorkshops/The-Data-Science-Workshop/master/Chapter12/Dataset/client.csv'

3. Using the read_csv method from the package pandas, load the 4 tables into 4 new variable called 'df_disp', 'df_trans', 'df_account' and 'df_client'. Specify the parameter sep=';' as this file is not separated by comma but semi-columns

In [3]:
df_disp = pd.read_csv(disp_url, sep=';')
df_trans = pd.read_csv(trans_url, sep=';')
df_account = pd.read_csv(account_url, sep=';')
df_client = pd.read_csv(client_url, sep=';')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


4. Print the first 5 rows of df_trans using the method .head():

In [4]:
df_trans.head()

,trans_id,account_id,date,type,operation,amount,balance,k_symbol,bank,account
0,695247,2378,930101,PRIJEM,VKLAD,700.0,700.0,NaN,NaN,NaN
1,171812,576,930101,PRIJEM,VKLAD,900.0,900.0,NaN,NaN,NaN
2,207264,704,930101,PRIJEM,VKLAD,1000.0,1000.0,NaN,NaN,NaN
3,1117247,3818,930101,PRIJEM,VKLAD,600.0,600.0,NaN,NaN,NaN
4,579373,1972,930102,PRIJEM,VKLAD,400.0,400.0,NaN,NaN,NaN


5. Print the shape of df_trans using the attribute .shape:

In [5]:
df_trans.shape

(1056320, 10)

6. Print the first 5 rows of df_account using the method .head():

In [6]:
df_account.head()

,account_id,district_id,frequency,date
0,576,55,POPLATEK MESICNE,930101
1,3818,74,POPLATEK MESICNE,930101
2,704,55,POPLATEK MESICNE,930101
3,2378,16,POPLATEK MESICNE,930101
4,2632,24,POPLATEK MESICNE,930102


7. Merge df_trans and df_account together using left join on the column 'account_id' and save the new DataFrame called 'df_trans_acc'

In [0]:
df_trans_acc = pd.merge(df_trans, df_account, how='left', on='account_id')

8. Print the shape of 'df_trans_acc'

In [8]:
df_trans_acc.shape

(1056320, 13)

9. Print the first 5 rows of df_disp using the method .head():

In [9]:
df_disp.head()

,disp_id,client_id,account_id,type
0,1,1,1,OWNER
1,2,2,2,OWNER
2,3,3,2,DISPONENT
3,4,4,3,OWNER
4,5,5,3,DISPONENT


We can see that the 'account_id' column doesn't contain unique identifier and this will add additional rows after the merge. Let's subset this DataFrame to only 'OWNER' type 

10. Subset 'df_disp' to only keep the rows with 'OWNER' as type and save the results in a new DataFrame called 'df_disp_owner'

In [0]:
df_disp_owner = df_disp[df_disp['type'] == 'OWNER']

11. Check the number of duplicates on the column 'account_id' of 'df_disp_owner' using the .duplicated() and .sum() methods

In [11]:
df_disp_owner.duplicated(subset='account_id').sum()

0

Great! All account ids are unique now. Let's merge the DataFrames now.

12. Merge 'df_trans_acc' and 'df_disp_owner' together using left join on the column 'account_id'. Save the new DataFrame called 'df_trans_acc_disp' and print its shape.

In [12]:
df_trans_acc_disp = pd.merge(df_trans_acc, df_disp_owner, how='left', on='account_id')
df_trans_acc_disp.shape

(1056320, 16)

13. Print the first 5 rows of df_client using the method .head()

In [13]:
df_client.head()

,client_id,birth_number,district_id
0,1,706213,18
1,2,450204,1
2,3,406009,1
3,4,561201,5
4,5,605703,5


14. Merge 'df_trans_acc_disp' and 'df_client' together using left join on the columns 'client_id' and 'district_id'. Save the new DataFrame called 'df_merged' and print its shape.

In [14]:
df_merged = pd.merge(df_trans_acc_disp, df_client, how='left', on=['client_id', 'district_id'])
df_merged.shape

(1056320, 17)

15. Print the columns names of df_merged using the attributes .columns

In [15]:
df_merged.columns

Index(['trans_id', 'account_id', 'date_x', 'type_x', 'operation', 'amount',
       'balance', 'k_symbol', 'bank', 'account', 'district_id', 'frequency',
       'date_y', 'disp_id', 'client_id', 'type_y', 'birth_number'],
      dtype='object')

16. Rename the columns 'date_x', 'type_x', 'date_y' and 'type_y' to respectively 'trans_date', 'trans_type', 'account_creation' and'client_type'

In [0]:
df_merged.rename(columns={'date_x': 'trans_date', 'type_x': 'trans_type', 'date_y':'account_creation', 'type_y':'client_type'}, inplace=True)

17. Print the first 5 rows of df_merged using the method .head()

In [17]:
df_merged.head()

,trans_id,account_id,trans_date,trans_type,operation,amount,balance,k_symbol,bank,account,district_id,frequency,account_creation,disp_id,client_id,client_type,birth_number
0,695247,2378,930101,PRIJEM,VKLAD,700.0,700.0,NaN,NaN,NaN,16,POPLATEK MESICNE,930101,2873,2873,OWNER,755324.0
1,171812,576,930101,PRIJEM,VKLAD,900.0,900.0,NaN,NaN,NaN,55,POPLATEK MESICNE,930101,692,692,OWNER,NaN
2,207264,704,930101,PRIJEM,VKLAD,1000.0,1000.0,NaN,NaN,NaN,55,POPLATEK MESICNE,930101,844,844,OWNER,NaN
3,1117247,3818,930101,PRIJEM,VKLAD,600.0,600.0,NaN,NaN,NaN,74,POPLATEK MESICNE,930101,4601,4601,OWNER,NaN
4,579373,1972,930102,PRIJEM,VKLAD,400.0,400.0,NaN,NaN,NaN,77,POPLATEK MESICNE,930102,2397,2397,OWNER,NaN


18. Print the data types of each column using the attribute .dtypes

In [18]:
df_merged.dtypes

trans_id              int64
account_id            int64
trans_date            int64
trans_type           object
operation            object
amount              float64
balance             float64
k_symbol             object
bank                 object
account             float64
district_id           int64
frequency            object
account_creation      int64
disp_id               int64
client_id             int64
client_type          object
birth_number        float64
dtype: object

The columns 'trans_date' and 'account_creation' are integer. We need to convert into datetime.

19. Convert the columns 'trans_date' and 'account_creation' using the .to_datetime() with the parameter format="%y%m%d"

In [0]:
df_merged['trans_date'] = pd.to_datetime(df_merged['trans_date'], format="%y%m%d")
df_merged['account_creation'] = pd.to_datetime(df_merged['account_creation'], format="%y%m%d")

20. Print the data types of each column using the attribute .dtypes

In [20]:
df_merged.dtypes

trans_id                     int64
account_id                   int64
trans_date          datetime64[ns]
trans_type                  object
operation                   object
amount                     float64
balance                    float64
k_symbol                    object
bank                        object
account                    float64
district_id                  int64
frequency                   object
account_creation    datetime64[ns]
disp_id                      int64
client_id                    int64
client_type                 object
birth_number               float64
dtype: object

We need to perform some transformations on the column 'birth_number' as it has a specific coding that includes date of birth of a person but also the sex.

More details on this coding can be found here:
https://data.world/lpetrocelli/czech-financial-dataset-real-anonymized-transactions/workspace/data-dictionary

21. Create a new column called 'is_female' by performing the following calculation to extract the sex information: (df_merged['birth_number'] % 10000) / 5000 > 1

In [0]:
df_merged['is_female'] = (df_merged['birth_number'] % 10000) / 5000 > 1

22. Print out the first 5 rows of the column 'birth_number'

In [22]:
df_merged['birth_number'].head()

0    755324.0
1         NaN
2         NaN
3         NaN
4         NaN
Name: birth_number, dtype: float64

23. Transform all the rows with 'is_female' is True by removing the value in 'birth_number' by 5000

In [0]:
df_merged.loc[df_merged['is_female'] == True, 'birth_number'] -= 5000

24. Print out the first 5 rows of the column 'birth_number'

In [24]:
df_merged['birth_number'].head()

0    750324.0
1         NaN
2         NaN
3         NaN
4         NaN
Name: birth_number, dtype: float64

25. Convert the 'birth_number' column with the .to_datetime() method with the following parameters: format="%y%m%d", errors='coerce'

In [25]:
pd.to_datetime(df_merged['birth_number'], format="%y%m%d", errors='coerce')

0         1975-03-24
1                NaT
2                NaT
3                NaT
4                NaT
5         2038-08-12
6                NaT
7         1979-03-24
8         1971-03-02
9                NaT
10        1970-06-24
11               NaT
12               NaT
13        2028-04-02
14        2040-12-02
15               NaT
16        2025-08-30
17               NaT
18               NaT
19        1978-06-27
20               NaT
21        1978-06-27
22        1971-04-26
23        2037-11-07
24               NaT
25               NaT
26        2022-12-17
27        2037-11-07
28        2021-06-17
29        1971-03-02
             ...    
1056290   2053-01-14
1056291   2028-05-17
1056292   2064-07-20
1056293   2022-08-20
1056294          NaT
1056295   2064-04-29
1056296   1979-03-02
1056297   2045-11-20
1056298   2049-12-15
1056299   2024-03-15
1056300   2054-12-08
1056301   2052-08-03
1056302   2052-01-25
1056303   2046-05-11
1056304   2061-08-20
1056305   1969-09-14
1056306   203

Because the year was recorded with only 2 digits in this dataset, the date are sometime converted to either 1900 or 2000 years. We need to fix this issue.

26. Convert the 'birth_number' column to string using .astype() method and print out the first 5 rows of the column 'birth_number'

In [26]:
df_merged['birth_number'] = df_merged['birth_number'].astype(str)
df_merged['birth_number'].head()

0    750324.0
1         nan
2         nan
3         nan
4         nan
Name: birth_number, dtype: object

After the conversion to string, all missing values have been converted to a string with value 'nan'. Let's convert them back to proper missing values

27. Import the numpy package and change the value of 'birth_number' for all rows with value 'nan' to np.nan. Print out the first 5 rows of the column 'birth_number'

In [27]:
import numpy as np
df_merged.loc[df_merged['birth_number'] == 'nan', 'birth_number'] = np.nan
df_merged['birth_number'].head()

0    750324.0
1         NaN
2         NaN
3         NaN
4         NaN
Name: birth_number, dtype: object

28. Add the prefix '19' to 'birth_number' for all rows that don't have missing values for this column

In [28]:
df_merged.loc[~df_merged['birth_number'].isna(), 'birth_number'] = '19' + df_merged.loc[~df_merged['birth_number'].isna(), 'birth_number']
df_merged['birth_number'].head()

0    19750324.0
1           NaN
2           NaN
3           NaN
4           NaN
Name: birth_number, dtype: object

29. Convert the 'birth_number' column with the .to_datetime() method with the following parameters: format="%y%m%d", errors='coerce' and save the results back to 'birth_number'. Print out the first 20 rows of the column 'birth_number'

In [29]:
df_merged['birth_number'] = pd.to_datetime(df_merged['birth_number'], format="%Y%m%d", errors='coerce')
df_merged['birth_number'].head(20)

0    1975-03-24
1           NaT
2           NaT
3           NaT
4           NaT
5    1938-08-12
6           NaT
7    1979-03-24
8    1971-03-02
9           NaT
10   1970-06-24
11          NaT
12          NaT
13   1928-04-02
14   1940-12-02
15          NaT
16   1925-08-30
17          NaT
18          NaT
19   1978-06-27
Name: birth_number, dtype: datetime64[ns]

Excellent we have fixed the year issue. We can now create a new feature that will calculate the age of the customer when his/her account was created

30. Create a new column called 'age_at_creation' by substracting 'account_creation' to 'birth_number'

In [0]:
df_merged['age_at_creation'] = df_merged['account_creation'] - df_merged['birth_number']

31. Convert the timedelta results in 'age_at_creation' by dividing it by np.timedelta64(1,'Y')

In [0]:
df_merged['age_at_creation'] = df_merged['age_at_creation'] / np.timedelta64(1,'Y')

32. Convert 'age_at_creation' to integer using the .round() method. Print the first 5 rows of df_merged

In [32]:
df_merged['age_at_creation'] = df_merged['age_at_creation'].round()
df_merged.head()

,trans_id,account_id,trans_date,trans_type,operation,amount,balance,k_symbol,bank,account,district_id,frequency,account_creation,disp_id,client_id,client_type,birth_number,is_female,age_at_creation
0,695247,2378,1993-01-01,PRIJEM,VKLAD,700.0,700.0,NaN,NaN,NaN,16,POPLATEK MESICNE,1993-01-01,2873,2873,OWNER,1975-03-24,True,18.0
1,171812,576,1993-01-01,PRIJEM,VKLAD,900.0,900.0,NaN,NaN,NaN,55,POPLATEK MESICNE,1993-01-01,692,692,OWNER,NaT,False,NaN
2,207264,704,1993-01-01,PRIJEM,VKLAD,1000.0,1000.0,NaN,NaN,NaN,55,POPLATEK MESICNE,1993-01-01,844,844,OWNER,NaT,False,NaN
3,1117247,3818,1993-01-01,PRIJEM,VKLAD,600.0,600.0,NaN,NaN,NaN,74,POPLATEK MESICNE,1993-01-01,4601,4601,OWNER,NaT,False,NaN
4,579373,1972,1993-01-02,PRIJEM,VKLAD,400.0,400.0,NaN,NaN,NaN,77,POPLATEK MESICNE,1993-01-02,2397,2397,OWNER,NaT,False,NaN


Well done! In this activity, we created new features by merging different tables together and manipulating date columns. We have now a much richer dataset with extra valuable information that can be fed to a Machine Learning model.